In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr5051
Linux


In [3]:
path = hdfs_home+"/dataSets/spark-guide/activity-data/"

In [4]:
spark_home = "/usr/hdp/current/spark2-client"

In [5]:
import findspark
findspark.init(spark_home)
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg23-streamingInProduction").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg23-streamingInProduction").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://ip-172-31-35-141.ec2.internal:8088/proxy/application_1534772926501_6711


In [9]:
static = spark.read.json(path)

In [10]:
streaming = spark\
  .readStream\
  .schema(static.schema)\
  .option("maxFilesPerTrigger", 10)\
  .json(path)\
  .groupBy("gt")\
  .count()

In [11]:
query = streaming\
  .writeStream\
  .outputMode("complete")\
  .option("checkpointLocation", "/tmp/")\
  .queryName("test_python_stream")\
  .format("memory")\
  .start()


# COMMAND ----------

In [12]:
from time import sleep
for x in range(3):
    spark.sql("SELECT * FROM test_python_stream").show(5, False)
    sleep(5)

+---+-----+
|gt |count|
+---+-----+
+---+-----+

+---+-----+
|gt |count|
+---+-----+
+---+-----+

+--------+------+
|gt      |count |
+--------+------+
|stairsup|104521|
|sit     |123085|
|stand   |113849|
|walk    |132560|
|bike    |107974|
+--------+------+
only showing top 5 rows



In [13]:
spark.streams.active